In [ ]:
import os
import cv2
import ipywidgets.widgets as widgets
import threading
import time
import tkinter as tk
from tkinter import Label
from Smart_Car import SG90
image_widget = widgets.Image(format='jpeg', width=640, height=480) #设置摄像头显示组件 Set up the camera display component
#bgr8转jpeg格式 bgr8 to jpeg format
import enum
import cv2
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])
image = cv2.VideoCapture(0) #打开摄像头/dev/video0 Open the camera /dev/video0
# 检查摄像头是否成功打开
if not image.isOpened():
    raise IOError("Cannot open webcam")
width=640
height=480
image.set(cv2.CAP_PROP_FRAME_WIDTH,width)#设置图像宽度 Set the image width
image.set(cv2.CAP_PROP_FRAME_HEIGHT,height)#设置图像高度 Set the image height
# image.set(3,600)
# image.set(4,500)
# image.set(5, 30) #设置帧率 Setting the frame rate
# image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 40) #设置亮度 -64 - 64 0.0 Set Brightness -64 - 64 0.0
# image.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64 2.0 Set Contrast -64 - 64 2.0
# image.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000 156.0 Set the exposure value 1.0 - 5000 156.0
ret, frame = image.read() #读取摄像头数据 Reading camera data
image_widget.value = bgr8_to_jpeg(frame)

def camera():
    try:
        display(image_widget) #显示摄像头组件 Display camera components
        while 1:
            ret, frame = image.read()
            image_widget.value = bgr8_to_jpeg(frame)
            time.sleep(0.010)
    except KeyboardInterrupt:
        print(" Program closed! ")
        image.release()
        pass

# 初始化到中心位置
reset = widgets.Button(description="Reset")
display(reset)
def on_button_clickedreset(b):
    global angle_up, angle_down
    angle_up = 90
    angle_down = 95
    SG90.set_angleup(angle_up)
    SG90.set_angledown(angle_down)

reset.on_click(on_button_clickedreset)
on_button_clickedreset(0)

video = widgets.Button(description="Video")
display(video)
# 视频录制状态
recording = False
out = None

# 开始/停止录像的函数
def on_button_clickedvideo(b):
    global recording, out
    if not recording:
        # 开始录像
        timestamp = int(time.time())
        file_name = os.path.join('/home/asus/Webcam', f"video_{timestamp}.avi")
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(file_name, fourcc, 20.0, (width, height))
        recording = True
    else:
        # 停止录像
        out.release()
        recording = False
video.on_click(on_button_clickedvideo)

photo = widgets.Button(description="Photo")
display(photo)
# 拍照的函数
def on_button_clickedphoto(b):
    ret, frame = image.read()
    if ret:
        timestamp = int(time.time())
        file_name = os.path.join('/home/asus/Webcam', f"photo_{timestamp}.jpg")
        cv2.imwrite(file_name, frame)
photo.on_click(on_button_clickedphoto)


Flagup = False
w = widgets.Button(description="W")
display(w)
def on_button_clickedw(b):
    global Flagup, angle_up
    Flagup = not Flagup
    if Flagup:
        angle_up -= 5
        if angle_up < 0:
            angle_up = 0
        SG90.set_angleup(angle_up)
        Flagup = not Flagup

w.on_click(on_button_clickedw)

s = widgets.Button(description="S")
display(s)
def on_button_clickeds(b):
    global Flagup, angle_up
    Flagup = not Flagup
    if Flagup:
        angle_up += 5
        if angle_up > 180:
            angle_up = 180
        SG90.set_angleup(angle_up)
        Flagup = not Flagup

s.on_click(on_button_clickeds)

Flagdown = False
a = widgets.Button(description="A")
display(a)
def on_button_clickeda(b):
    global Flagdown, angle_down
    Flagdown = not Flagdown
    if Flagdown:
        angle_down += 5
        if angle_down > 180:
            angle_down = 180
        SG90.set_angledown(angle_down)
        Flagdown = not Flagdown

a.on_click(on_button_clickeda)

d = widgets.Button(description="D")
display(d)
def on_button_clickedd(b):
    global Flagdown, angle_down
    Flagdown = not Flagdown
    if Flagdown:
        angle_down -= 5
        if angle_down < 0:
            angle_down = 0
        SG90.set_angledown(angle_down)
        Flagdown = not Flagdown

d.on_click(on_button_clickedd)

camera_thread = threading.Thread(target = camera)
camera_thread.start()
